In [1]:
import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
#import Utils.DiaSourceTools as DSTools
import lsst.afw.display.ds9 as ds9
from astropy.time import Time

import re
from astropy.table import Table
import astropy.coordinates as coord
import astropy.units as u


In [2]:
DATADIR="/sps/lsst/data/fouchez/DC2/testCB/tst2/output"

In [3]:
directory = DATADIR
butler = dafPersist.Butler(directory)

/sps/lsst/software/lsst_distrib/w_2018_18/python/miniconda3-4.3.21/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
deepDiff = butler.get("deepDiff_differenceExp", {"filter":'r', "raft":'2,2',"sensor":'1,0', "visit":417009})

In [5]:
diasrc = butler.get("deepDiff_diaSrc", {"filter":'r', "raft":'2,2',"sensor":'1,0', "visit":417009})

In [6]:
visits=butler.queryMetadata('deepDiff_diaSrc', ['visit'], dataId={'filter': 'r'})

In [7]:
visits = np.array(visits)
mask = (visits >= 417009) & (visits <= 417057)
visits =  visits[mask]
print (visits, len(visits))

[417009 417057] 2


In [8]:
rafts = ["2,2"]
#dates = ['2006-05-20','2006-05-24']

multi_matches = None
for visit in visits:
    date_catalog = None
    for raft in rafts:
        print (raft,visit)
        print (butler.datasetExists("deepDiff_diaSrc", {"raft":raft,"sensor":"1,0","filter":'r', "visit":visit}))
        if butler.datasetExists("deepDiff_diaSrc", {"raft":raft,"filter":'r', "sensor":"1,0", "visit":visit}) == True:
            catalog = butler.get("deepDiff_diaSrc", {"raft":raft,"filter":'r', "sensor":"1,0", "visit":visit})
            if date_catalog is None: 
                date_catalog = catalog.copy()
            else:
                date_catalog.extend(catalog)
                
    if multi_matches is None and len(date_catalog)>0:
        multi_matches = afwTable.MultiMatch(date_catalog[0].schema, {'date':float})
    
    #t = Time(date)
    multi_matches.add(date_catalog, {'date':visit})
                
results = multi_matches.finish(removeAmbiguous=False)


2,2 417009
True
2,2 417057
True


In [12]:
def threshold_light_curves(light_curves, threshold):
    t_light_curves = [lc for lc in light_curves if len(lc) >= threshold]
    return t_light_curves

def build_lightcurve(source_list):
    """
    Assemble a light curve data table from available files.
    """

    bandpasses = ['r','i']


    lightcurve = {}
    lightcurve['classification'] = []
    lightcurve['bandpass'] = []
    lightcurve['mjd'] = []
    lightcurve['ra'] = []
    lightcurve['dec'] = []
    lightcurve['flux'] = []
    lightcurve['flux_error'] = []
    lightcurve['zp'] = []
    lightcurve['zpsys'] = []


    for mjd, src in source_list:

        #print 'yep',visit
        lightcurve['classification'].append(src['ip_diffim_ClassificationDipole_value'])
        lightcurve['bandpass'].append(str('sdss' + bandpasses[0]))
        lightcurve['mjd'].append(mjd)
        lightcurve['ra'].append(src['coord_ra'])
        lightcurve['dec'].append(src['coord_dec'])
        lightcurve['flux'].append(src['base_CircularApertureFlux_4_5_flux'])
        lightcurve['flux_error'].append(src['base_CircularApertureFlux_4_5_fluxSigma'])
        lightcurve['zp'].append(25.0)
        lightcurve['zpsys'].append('ab')
    lightcurve = Table(data=lightcurve)
    return lightcurve


In [10]:
light_curves = []
i = 0
current = -1
while i < len(results):
    result = results[i]
    if current == -1 or current != result['object']:
        lc = [(result['date'],result)]
        light_curves.append(lc)
        current = result['object']
    else:
        light_curves[-1].append((result['date'],result))
    i+=1

print (len(light_curves))
t_light_curves = threshold_light_curves(light_curves, 2)
print (len(t_light_curves))

4464
210


In [13]:
lcs = []
for light_curve in t_light_curves:
    lcs.append(build_lightcurve(light_curve))

In [15]:
PI= 3.141592   
for i, lc in enumerate(lcs):
       print(i,"%2.8f"% (lc[0]['ra']/PI*180),"%2.8f"% (lc[0]['dec']/PI*180),' ',lc[0]['flux'],lc[1]['flux'])
    


0 55.39093366 -27.38991028   -1296.05407715 -956.209777832
1 55.44892494 -27.42755175   -4661.55273438 -3966.29858398
2 55.46389755 -27.43363622   -1166.85107422 -1056.67248535
3 55.40046465 -27.41174587   -1461.125 -713.429199219
4 55.51288668 -27.46329216   1918.04504395 622.590393066
5 55.51010627 -27.46240300   3663.34594727 2164.03930664
6 55.51739197 -27.46537220   1186.19897461 770.931335449
7 55.37158134 -27.40817929   5416.40527344 293.110748291
8 55.53165874 -27.47298491   144818.65625 114330.851562
9 55.36369074 -27.40945010   28713.1230469 23295.9003906
10 55.51056563 -27.47433766   4257.0546875 2956.66479492
11 55.52671068 -27.48179186   7472.2109375 5024.94580078
12 55.50637000 -27.47379744   2948.24145508 1779.22131348
13 55.50159534 -27.47265248   1034.82519531 797.195922852
14 55.49867028 -27.47268670   3534.29882812 2230.55981445
15 55.52726182 -27.48434178   33665.734375 nan
16 55.42052493 -27.44167482   4681.8671875 2767.57275391
17 55.50675977 -27.47717928   4263.1